In [107]:
import asyncio
import json
from datetime import datetime
import requests
import hashlib
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q

import requests
import web3
import pandas as pd

import sys
sys.path.append("..")

import chainklik.libs.common.utils as utils
import chainklik.libs.aave.api as aave_api
import chainklik.libs.common.payload as payload
import chainklik.libs.common.etherscan as etherscan
import eth_abi
import chainklik.config.config as cfg

import warnings
warnings.filterwarnings('ignore')

In [108]:
es = Elasticsearch(
    "https://localhost:9200",
    http_auth=('elastic', 'y=fUp=8ucKL18I5K=1Am'),
    verify_certs=False
)

In [109]:
def es_create_index(index_name):
    try:
        es.indices.create(index=index_name)
    except Exception as e:
        print(e)

In [414]:
index_name = 'abi'
es_create_index(index_name)
es.indices.delete(index=index_name, ignore=[400, 404])

BadRequestError(400, 'resource_already_exists_exception', 'index [abi/UvZUJ7DfRwylzhasMlXvrA] already exists')


ObjectApiResponse({'acknowledged': True})

In [415]:
es.indices.create(
    index=index_name,
    body={
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0,

            "analysis": {
                "analyzer": {
                    "lc_analyzer": {
                        "tokenizer": "lowercase",
                    },
                    "addr_analyzer" : {
                        "tokenizer": "edge_ngram_tokenizer"
                    }
                },
                "tokenizer": {
                    "edge_ngram_tokenizer": {
                        "type": "edge_ngram",
                        "min_gram": 8,
                        "max_gram": 42,
                        "token_chars": ["letter", "digit"]
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "name": {
                    "type": "text",
                    "analyzer": "lc_analyzer" 
                },
                "contract": {
                    "type": "text",
                    "analyzer": "lc_analyzer"
                },
                "type": {
                    "type": "text",
                    "analyzer": "lc_analyzer"
                },
                "desc": {
                    "type": "text",
                    "analyzer": "lc_analyzer"
                },
                "source_code": {
                    "type": "text",
                    "analyzer": "lc_analyzer"
                },
                "address": {
                    "type": "text",
                    "analyzer": "addr_analyzer"
                }
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'abi'})

In [416]:
# use aave v2 as an example
ws = web3.Web3(web3.Web3.HTTPProvider(cfg.config["http_url"]))

In [417]:
def doc_id(chain, contract_addr, abi):
    return hashlib.md5('{}/{}/{}'.format(chain,contract_addr,str(abi)).encode()).hexdigest()

In [418]:
def add_abi(chain, contract_addr, impl_addr, contract_name, source_code_link, author = "rshi"):
    # if contract addr is different from impl addr
    pool_contract = etherscan.get_contract(contract_addr,impl_addr)
    # save contract abi
    contract_id = doc_id(chain, contract_addr, pool_contract.abi)
    doc = {
        "author": "rshi",
        "timestamp": datetime.now(),
        "id" : contract_id,
        "contract_id" : contract_id,
        "chain" : chain,
        "address" : contract_addr,
        "contract" : contract_name,
        "type" : "contract",
        "name" : contract_name,
        "abi": pool_contract.abi,
        "desc": contract_name + " contract",
        "source_code" : source_code_link
    }
    resp = es.index(index='abi', id=doc['id'], document=doc)

    # save contract function and event abi
    for abi in pool_contract.abi:
        if abi["type"] not in ["constructor","fallback","receive"]:
            doc = {
                "author": "rshi",
                "timestamp": datetime.now(),
                "id" : doc_id(chain, contract_addr, abi),
                "contract_id" : contract_id,
                "chain" : chain,
                "address" : contract_addr,
                "contract" : contract_name,
                "type" : abi["type"],
                "name" : abi["name"],
                "abi": abi,
                "desc": " ".join([contract_name, abi["name"], abi["type"]]),
                "source_code" : source_code_link
            }
            resp = es.index(index='abi', id=doc['id'], document=doc)

In [420]:
chain = "ethereum"
contract_addr = "0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"
impl_addr = contract_addr
contract_name = "Curve 3pool"
link = "https://github.com/curvefi/curve-contract"
add_abi(chain, contract_addr, impl_addr, contract_name, link)

In [421]:
chain = "ethereum"
contract_addr = "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9"
impl_addr = "0x085E34722e04567Df9E6d2c32e82fd74f3342e79"
contract_name = "Aave v2"
link = "https://github.com/aave/protocol-v2"
add_abi(chain, contract_addr, impl_addr, contract_name, link)

In [401]:
s = Search(using=es, index=index_name)
# keywords = "0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9"
# keyword_queries = []
    
# for keyword in keywords:
#     keyword_queries.append(Q("multi_match", query=keyword, fields=["_all"]))

# combined_query = Q('bool', should=keyword_queries)
# s = s.query(combined_query)

# query = Q('bool',
#     must=[
#         Q('match', address="0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"),
#         Q('match', type='function'), 
#         Q('match', name='get_dy')
#     ]
# )

page_size = 50  # Number of results per page
page_number = 1  # Page number (starting from 1)

# Calculate the starting point (from) based on page size and page number
from_index = (page_number - 1) * page_size

# Set the pagination parameters in the search query
s = s[from_index:from_index + page_size]

query = Q('bool',
    should=[
        Q('multi_match', query="0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7", fields=["address"]),
        Q('multi_match', query="remove_liquidity_one_coin", fields=["name","contract","type"])
    ]
)

s = s.query(query)

response = s.execute()

response.to_dict()["hits"]["hits"][0]

{'_index': 'abi',
 '_id': '793595436e9a0d313ada8ea503e58f59',
 '_score': 28.591627,
 '_source': {'author': 'rshi',
  'timestamp': '2023-12-05T04:06:27.692053',
  'id': '793595436e9a0d313ada8ea503e58f59',
  'contract_id': '58aece5b6572a987bde6a09ac8ab5f39',
  'address': '0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7',
  'contract': 'Curve 3pool',
  'type': 'function',
  'name': 'remove_liquidity_one_coin',
  'abi': {'name': 'remove_liquidity_one_coin',
   'outputs': [],
   'inputs': [{'type': 'uint256', 'name': '_token_amount'},
    {'type': 'int128', 'name': 'i'},
    {'type': 'uint256', 'name': 'min_amount'}],
   'stateMutability': 'nonpayable',
   'type': 'function',
   'gas': 4025523},
  'desc': 'Curve 3pool remove_liquidity_one_coin function',
  'source_code': 'https://github.com/curvefi/curve-contract'}}

In [408]:
from elasticsearch.client import IndicesClient

indices_client = IndicesClient(es)

# Use the _analyze endpoint to test the custom analyzer
analysis_results = indices_client.analyze(
    index=index_name,
    body={
        "analyzer": "lc_analyzer",
        "text": "https://github.com/Uniswap/v3-core"
    }
)

In [409]:
analysis_results

ObjectApiResponse({'tokens': [{'token': 'https', 'start_offset': 0, 'end_offset': 5, 'type': 'word', 'position': 0}, {'token': 'github', 'start_offset': 8, 'end_offset': 14, 'type': 'word', 'position': 1}, {'token': 'com', 'start_offset': 15, 'end_offset': 18, 'type': 'word', 'position': 2}, {'token': 'uniswap', 'start_offset': 19, 'end_offset': 26, 'type': 'word', 'position': 3}, {'token': 'v', 'start_offset': 27, 'end_offset': 28, 'type': 'word', 'position': 4}, {'token': 'core', 'start_offset': 30, 'end_offset': 34, 'type': 'word', 'position': 5}]})